# Collaborative Filtering for Implicit Feedback (Last.fm 500)

**Goal.** Re-implementation of *Implicit ALS* (Hu, Koren, Volinsky, 2008) on a reduced Last.fm dataset (~500 users), evaluating **novelty recovery** (re-watches removed).


## 0. Import Libraries
This cell imports the required Python libraries for data manipulation and numerical computations.


In [1]:
# === Setup & Imports ===
import os
import random
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import timedelta

# SciPy is used only for sparse matrices (CSR/CSC)
from scipy import sparse

# Fix random seeds for reproducibility
RNG_SEED = 24
np.random.seed(RNG_SEED)
random.seed(24)
os.environ["PYTHONHASHSEED"] = "24"

# 1. Load and Clean Last.fm Datasets

In this step we load the reduced Last.fm datasets:

For reproducibility and runtime feasibility we restrict the dataset to the first 500 users. This reduction may affect optimal hyperparameters compared to the original paper.

- **Profiles**: user demographic information (gender, age, country, signup date).  
- **Listens**: implicit feedback records (user listening history).

We produce two clean DataFrames: profiles and listens, ready for analysis, applying some data cleaning.

In [2]:
# === Load CSVs (profiles + listens) ===

PROFILES_CSV = Path("Datasets-lastfm-reduced/profiles_first500.csv")
LISTENS_CSV  = Path("Datasets-lastfm-reduced/listens_first500_CAP.csv")

profiles = pd.read_csv(PROFILES_CSV)
listens  = pd.read_csv(LISTENS_CSV)

display(profiles.head())
display(listens.head())

,user_id,gender,age,country,signup
0,#id,gender,age,country,registered
1,user_000001,m,NaN,Japan,"Aug 13, 2006"
2,user_000002,f,NaN,Peru,"Feb 24, 2006"
3,user_000003,m,22,United States,"Oct 30, 2005"
4,user_000004,f,NaN,NaN,"Apr 26, 2006"


,user_id,timestamp,artist_id,artist_name,track_id,track_name
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


In [3]:
# === Clean & preprocess data ===
# Clean profiles: skips first row that repeats header names
profiles = pd.read_csv(PROFILES_CSV,header=1)

# Ensure timestamp is parsed as UTC
listens["timestamp"] = pd.to_datetime(listens["timestamp"], utc=True, errors="coerce")

#rename columns for consistency
profiles = profiles.rename(columns={"#id": "user_id", "registered": "signup"}) 

# We work at ARTIST level to avoid NaNs in track_id
listens = listens.rename(columns={"artist_id": "item_id", "artist_name": "item_name"})
# Drop rows with NaN item_id
listens = listens[listens["item_id"].notna()].copy() 
listens = listens[listens["timestamp"] <= "2009-06-20"].copy()  # removed outlier value (2010)

print("Profiles shape:", profiles.shape)
print("Listens shape:", listens.shape)
display(profiles.head())
display(listens.head())


Profiles shape: (499, 5)
Listens shape: (1986196, 6)


,user_id,gender,age,country,signup
0,user_000001,m,NaN,Japan,"Aug 13, 2006"
1,user_000002,f,NaN,Peru,"Feb 24, 2006"
2,user_000003,m,22.0,United States,"Oct 30, 2005"
3,user_000004,f,NaN,NaN,"Apr 26, 2006"
4,user_000005,m,NaN,Bulgaria,"Jun 29, 2006"


,user_id,timestamp,item_id,item_name,track_id,track_name
0,user_000001,2009-05-04 23:08:57+00:00,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04 13:54:10+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04 13:52:04+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04 13:42:52+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04 13:42:11+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


# 2. Train/Test split by time

We divide the dataset into a training set and a test set, using the last 30 days of listening events as the test period.

In [4]:
# === Train/Test split by time ===

def split_train_test_time(df: pd.DataFrame, test_days: int = 7):
    """
    Split interactions into train/test by time (global cutoff).
    Train = all interactions before cutoff.
    Test  = all interactions after cutoff.
    """
    cutoff = df["timestamp"].max() - pd.Timedelta(days=test_days)
    train_df = df[df["timestamp"] <= cutoff].copy()
    test_df  = df[df["timestamp"]  > cutoff].copy()
    return train_df, test_df

train_df, test_df = split_train_test_time(listens, test_days=30)

print("Train listens:", train_df.shape, " Test listens:", test_df.shape)
display(train_df.head(), test_df.head())

Train listens: (1943981, 6)  Test listens: (42215, 6)


,user_id,timestamp,item_id,item_name,track_id,track_name
0,user_000001,2009-05-04 23:08:57+00:00,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04 13:54:10+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04 13:52:04+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04 13:42:52+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04 13:42:11+00:00,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


,user_id,timestamp,item_id,item_name,track_id,track_name
47454,user_000011,2009-06-17 17:00:58+00:00,2f3dfafb-be37-4d6c-86c7-23d0650497d4,Laineen Kasperi,31459b6a-c642-4786-b102-d42377f7922e,Järjen Häivii
47455,user_000011,2009-06-08 11:53:00+00:00,d2f84ebe-ce01-4612-9b1d-cd016e07cb88,Flash,NaN,Open Sky
47456,user_000011,2009-06-06 15:30:41+00:00,bd13909f-1c29-4c27-a874-d4aaf27c5b1a,Fleetwood Mac,1988e204-96d5-49b0-bf73-450c572780d4,Black Magic Woman
47457,user_000011,2009-06-05 22:03:46+00:00,bd13909f-1c29-4c27-a874-d4aaf27c5b1a,Fleetwood Mac,4cbbb2f9-b5d1-43ce-8aea-b281095a9564,Stop Messin' Round
47458,user_000011,2009-06-05 22:00:54+00:00,bd13909f-1c29-4c27-a874-d4aaf27c5b1a,Fleetwood Mac,cb47245d-0489-4b59-b203-378a2221b2fb,Looking For Somebody


# 3. Remove Re-Watched Items from Test Set

Following Hu et al. (2008), we remove from the test set all (user, item) pairs that already appeared in the training set.  

This ensures that the evaluation focuses on **novelty** (first-time recommendations) instead of rewarding trivial re-listening, which would artificially inflate the metrics.

In [5]:
# === Remove “re-watches” from test (optional, as in paper) ===
def remove_rewatched_from_test(train_df: pd.DataFrame, test_df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove from test the (user,item) pairs that appear in train,
    to focus evaluation on discovery (first-time recommendations).
    """

    # Keep only the unique (user,item) pairs from the training set
    train_pairs = train_df[["user_id", "item_id"]].drop_duplicates()

    # Merge test with train pairs to flag which interactions already existed in train
    merged = test_df.merge(train_pairs, on=["user_id", "item_id"], how="left", indicator=True)

    # Keep only the test interactions that do not appear in train ("left_only")
    filtered = merged[merged["_merge"] == "left_only"].drop(columns=["_merge"])

    return filtered.reset_index(drop=True)  # Return the filtered test set, reindexed for cleanliness

test_df_filtered = remove_rewatched_from_test(train_df, test_df)
print("Test listens (filtered):", test_df_filtered.shape)


Test listens (filtered): (13512, 6)


Test set reduced to 13.512 interactions (from 42.214); still sufficient for meaningful Recall@K evaluation.


# 4. Build User–Item Utility matrix

Map user/item IDs to indices and build sparse CSR matrices (`train_matrix`, `test_matrix`) with per-(user, item) listen counts.


In [6]:
# === Build ID maps from TRAIN and CSR Utilty matrices ===

# Transform user/item ids to contiguous indices
user_ids = profiles["user_id"].tolist()
item_ids = sorted(train_df["item_id"].unique().tolist())  # items seen in TRAIN only

# Create dictionaries for id
user2idx = {u: i for i, u in enumerate(user_ids)} # user-id to index mapping
item2idx = {it: i for i, it in enumerate(item_ids)} # item-id to index mapping

# Inverse mappings
idx2user = {i: u for u, i in user2idx.items()}
idx2item = {i: it for it, i in item2idx.items()}

def df_to_csr_counts(df: pd.DataFrame,
                     user2idx: dict,
                     item2idx: dict,
                     shape: tuple | None = None) -> sparse.csr_matrix:
    """
    Build a CSR user-item matrix with counts of interactions.
    Rows = users
    Cols = items
    Data = counts.
    """

    # count how many times each (user, item) pair appears
    grouped = df.groupby(["user_id", "item_id"]).size().reset_index(name="count")

    # map user_id and item_id to integer indices
    grouped["row"] = grouped["user_id"].map(user2idx)
    grouped["col"] = grouped["item_id"].map(item2idx) #

    # drop rows where either user or item is not in the dictionaries (cold-start cases)
    grouped = grouped.dropna(subset=["row", "col"]) 

    # extract rows, cols, and counts as NumPy arrays
    rows = grouped["row"].astype(int).to_numpy()
    cols = grouped["col"].astype(int).to_numpy() 
    data = grouped["count"].astype(np.float32).to_numpy() 

    # build the sparse CSR matrix
    n_users, n_items = len(user2idx), len(item2idx)
    if not shape:
        shape = (n_users, n_items)
    return sparse.csr_matrix((data, (rows, cols)), shape=shape, dtype=np.float32)

# Build utility matrices (users x items, counts)
train_matrix = df_to_csr_counts(train_df, user2idx, item2idx)
test_matrix  = df_to_csr_counts(test_df_filtered, user2idx, item2idx, shape=train_matrix.shape) 

n_users, n_items = train_matrix.shape
density = train_matrix.nnz / (n_users * n_items) * 100

print(f"CSR shapes -> train: {train_matrix.shape}, test: {test_matrix.shape}, nnz(train)={train_matrix.nnz}, density={density:.4f}%")


CSR shapes -> train: (499, 46736), test: (499, 46736), nnz(train)=204425, density=0.8766%


**Utility Matrix Sparsity**

The training matrix has shape **(499 × 46.736)**, meaning ~23.3 million possible user–item pairs.  
Out of these, number of non-zero entries are **204.425**, i.e. real user–item interactions.  

This yields a density of ~**0.9%**, confirming that the utility matrix is **highly sparse**. 
Such sparsity is typical of recommender system datasets and motivates the use of **sparse matrix representations** and **collaborative filtering methods** designed for sparse data.


## 5. Baseline: Global Popularity

As a simple non-personalized baseline, we recommend the globally most popular items:  
- Compute total counts per item in the training set.  
- For each user, mask already-seen items.  
- Recommend the top-K unseen items.  

This provides a lower-bound reference for evaluation: any personalized method should 
outperform the popularity baseline.

In [7]:
# === Baseline: Global Popularity ===

def get_seen_items(csr_matrix: sparse.csr_matrix, u: int) -> np.ndarray:
    """
    Return the indices of items with non-zero interactions for user u.
    Uses CSR row pointers (indptr) to quickly extract the indices.
    """
    start, end = csr_matrix.indptr[u], csr_matrix.indptr[u+1]
    return csr_matrix.indices[start:end]

def baseline_popularity_recall_at_k(train_csr: sparse.csr_matrix,
                                    test_csr: sparse.csr_matrix,
                                    K: int = 10) -> float:
    """
    Compute Recall@K for a popularity-based recommender.
    For each user u:
    - Recommend the K most popular items in train (excluding already-seen items).
    - Compare with test items to compute hits.
    """
    item_pop = np.asarray(train_csr.sum(axis=0)).flatten() # total counts per item
    popular_order = np.argsort(-item_pop)

    hits, total = 0, 0 # hit count and total relevant items 

    for u in range(train_csr.shape[0]):

        test_items = get_seen_items(test_csr, u) # items in test for user u
        if test_items.size == 0:
            continue # skip users with no test items
        
        # Extract items seen in train for user u
        seen = set(get_seen_items(train_csr, u).tolist()) # list of items consumed by user u in train
        reccomendations = [i for i in popular_order if i not in seen][:K] # top-K popular not seen
        hits += len(set(reccomendations).intersection(set(test_items.tolist())))
        total += len(test_items)
    return hits / total if total > 0 else np.nan

K = 10
pop_rec = baseline_popularity_recall_at_k(train_matrix, test_matrix, K=K)
print(f"[Popularity] Recall@{K}: {pop_rec:.4f}")


[Popularity] Recall@10: 0.0063


Recall@10 values are low due to dataset sparsity, but relative improvements (Popularity < Item–Item < ALS) remain consistent.

**Evaluation Metrics**

We evaluate recommendations using **ranking-based metrics** tailored for implicit feedback:

- **Recall@K**  
  Proportion of relevant test items appearing in the top-K recommendations.  
  *Higher is better* (0 = none recovered, 1 = all recovered).

- **MAP@K (Mean Average Precision)**  
  Accounts for both the presence and the ranking of relevant items.  
  Rewards models that rank relevant items earlier in the list.  
  *Higher is better.*

- **Expected Percentile Ranking (EPR)**  
  Average percentile rank of relevant test items across the full recommendation list.  
  *Lower is better* (≈50% = random ranking; closer to 0% = ideal ranking at the top).

**Summary:**  
- Recall@K → higher is better (“how many relevant items were recovered”)  
- MAP@K → higher is better (“how well they were ordered”)  
- EPR → lower is better (“how high they appear on average”)


## 6. Item–Item Collaborative Filtering (Cosine)

We implement a memory-based baseline using **cosine similarity** between item vectors:  
- Each item is represented as a vector of user interactions.  
- Cosine similarity measures co-consumption by the same users.  
- For each user, we score unseen items as the weighted sum of similarities with items already consumed.  

This method personalizes recommendations by leveraging similarity between items, but does not exploit confidence weighting, making it a simpler baseline compared to ALS.  

In [8]:
# === Baseline: Item-Item Collaborative Filtering (Cosine) ===

def item_item_cosine_scores(train_csr: sparse.csr_matrix) -> np.ndarray:
    """
    Compute item-item cosine similarity matrix S (I x I).
    """
    X = train_csr.T.tocsr().astype(np.float64)  # Transpose to items x users.
    
    #L2-normalize item rows
    norms = np.sqrt(X.multiply(X).sum(axis=1)).A1 + 1e-12 # avoid div by zero
    Xn = X.multiply(1.0 / norms[:, None]) # normalize rows
    S = Xn @ Xn.T # cosine similarity matrix
    return S

S = item_item_cosine_scores(train_matrix)

def cosine_recall_at_k(S: sparse.spmatrix | np.ndarray, train_csr, test_csr, K=10) -> float:
    """
    For each user u:
    - Build a binary item profile over seen train items.
    - Score candidates as the sum of cosine similarities to seen items.
    - Mask seen items, extract top-K, and count hits against test items.
    """

    hits, total = 0, 0
    for u in range(train_csr.shape[0]):
        seen = get_seen_items(train_csr, u)
        user_profile = np.zeros(S.shape[0], dtype=np.float64)
        user_profile[seen] = 1.0
        scores = user_profile @ S # score vector for affinity of user u to all items
        scores[seen] = -np.inf

        recs = np.argpartition(-scores, K)[:K]
        recs = recs[np.argsort(-scores[recs])]
        test_items = get_seen_items(test_csr, u)
        if test_items.size == 0:
            continue
        hits += len(set(recs).intersection(set(test_items.tolist())))
        total += len(test_items)
    return hits / total if total > 0 else np.nan

cos_rec = cosine_recall_at_k(S, train_matrix, test_matrix, K=10)
print(f"[Item-Item Cosine] Recall@{K}: {cos_rec:.4f}")

[Item-Item Cosine] Recall@10: 0.0094


## 7. Implicit ALS (Alternating Least Squares)

We implement the **ALS algorithm for implicit feedback** following Hu, Koren & Volinsky (2008).  

Key ideas:  
- Each user and item is represented by a latent factor vector.  
- Binary preferences \(p_{ui}\) are derived from implicit feedback (1 if user interacted, 0 otherwise).  
- Confidence weights \( c_{ui} = 1 + alpha * f(r_{ui} )\) scale the importance of frequent interactions.  
- The model alternates between updating user factors (X) and item factors (Y)  
  by solving regularized least-squares systems.  
- Variants:  
  - **Linear confidence**: \(f(r) = r\)  
  - **Log confidence**: \(f(r) = log(1 + r/c_0)\)  

This factorization captures hidden patterns beyond direct co-consumption, making ALS more powerful than memory-based methods such as Item–Item CF.


In [9]:
# === Implicit ALS implementation ===

class ImplicitALS:
    """
    Alternating Least Squares for Implicit Feedback
    Reference:
      Hu, Koren, Volinsky (2008), 'Collaborative Filtering for Implicit Feedback Datasets'
      - min_x,y sum_{u,i} c_ui (p_ui - x_u^T y_i)^2 + lambda (||x||^2 + ||y||^2)
      - p_ui = 1 if r_ui > 0 else 0
      - c_ui = 1 + alpha * r_ui   (or log-based)
    """
    def __init__(self, n_users, n_items, n_factors=64, n_iters=10, 
                 reg=0.1, alpha=40.0, use_log_conf=False, log_eps=1e-8):
        """
        n_users: number of users
        n_items: number of items
        n_factors: number of latent factors
        n_iters: number of ALS iterations
        reg: regularization parameter (lambda)
        alpha: confidence scaling factor
        use_log_conf: if True, use log-based confidence
        log_eps: small constant for log-based confidence
        """
        self.n_users = n_users
        self.n_items = n_items
        self.n_factors = n_factors
        self.n_iters = n_iters
        self.reg = reg
        self.alpha = alpha
        self.use_log_conf = use_log_conf
        self.log_eps = log_eps
        # Factors (initialized with small random values)
        self.X = 0.01 * np.random.randn(n_users, n_factors) # user factors
        self.Y = 0.01 * np.random.randn(n_items, n_factors) # item factors

    def _conf_from_counts(self, counts: np.ndarray) -> np.ndarray:
        """
        Compute confidence from counts:
        c_ui = 1 + alpha * f(r_ui).
        
        or for log-based:
        c_ui = 1 + alpha * log(1 + r_ui / c0),
        c0 is the mean of counts, used to normalize them.
        """
        if self.use_log_conf:
            
            c0 = max(1.0, np.mean(counts))
            return 1.0 + self.alpha * np.log(1.0 + counts / c0)
        else:
            return 1.0 + self.alpha * counts

    def fit(self, train_counts: sparse.csr_matrix):
        """
        Train ALS alternating users/items updates.
        train_counts: CSR matrix (#users x #items).
        """
        train_counts_csr_items = train_counts.tocsr() # train_counts in CSR (compressed sparse row)
        train_counts_csc_users = train_counts.tocsc() # train_counts in CSC (compressed sparse column)

        Id_mat = np.eye(self.n_factors, dtype=np.float64) # Identity matrix (f,f)

        # Main ALS loop
        for it in range(self.n_iters):

            # Precompute Y^T Y and X^T X once per half-iteration
            YtY = self.Y.T @ self.Y  # matricial product

            # === Update user factors ===
            for u in range(self.n_users):
                row_start, row_end = train_counts_csr_items.indptr[u], train_counts_csr_items.indptr[u+1]
                item_idx = get_seen_items(train_counts_csr_items, u) # list of items consumed by user u
                
                if item_idx.size == 0:
                    # cold user
                    A = YtY + self.reg * Id_mat
                    b = np.zeros(self.n_factors, dtype=np.float64)
                    self.X[u, :] = np.linalg.solve(A, b)
                    continue
                
                r_u = train_counts_csr_items.data[row_start:row_end]  # lists of counts for items consumed by user u
                c_u = self._conf_from_counts(r_u)               # confidence values
                c_u_minus1 = c_u - 1.0
                Y_u = self.Y[item_idx, :]                       # factors of items consumed by user u
                
                # A = Y^T Y + Y_u^T * diag(c_u - 1) * Y_u + reg*I
                A = YtY + (Y_u.T * c_u_minus1) @ Y_u + self.reg * Id_mat
                
                # b = Y_u^T * (c_u * p_u) ;  p_u = 1 for items seen by user u
                weighted_u = Y_u * c_u.reshape(-1,1)
                b = weighted_u.sum(axis=0)

                # update user factors
                self.X[u, :] = np.linalg.solve(A, b) # solve for x_u

            # === Update item factors ===

            XtX = self.X.T @ self.X # matricial product

            for i in range(self.n_items):
                col_start, col_end = train_counts_csc_users.indptr[i], train_counts_csc_users.indptr[i+1]
                user_idx = train_counts_csc_users.indices[col_start:col_end] # list of users that consumed item i
                
                if user_idx.size == 0:
                    # cold item
                    A = XtX + self.reg * Id_mat
                    b = np.zeros(self.n_factors, dtype=np.float64)
                    self.Y[i, :] = np.linalg.solve(A, b)
                    continue
                r_i = train_counts_csc_users.data[col_start:col_end]  # lists of counts for users that consumed item i
                c_i = self._conf_from_counts(r_i)        # confidence values
                c_i_minus1 = c_i - 1.0
                X_i = self.X[user_idx, :]                # factors of users that consumed item i

                # A = X^T X + X_i^T * diag(c_i - 1) * X_i + reg*I                      
                A = XtX + (X_i.T * c_i_minus1) @ X_i + self.reg * Id_mat

                # b = X_i^T * (c_i * p_i) ;  p_i = 1 for users that consumed item i
                weighted_i = X_i * c_i.reshape(-1,1)
                b = weighted_i.sum(axis=0)

                # update item factors
                self.Y[i, :] = np.linalg.solve(A, b)


    def recommend_for_user(self, u: int, K: int = 10, 
                           seen_items: np.ndarray | None = None) -> np.ndarray:
        """
        Return top-K item indices for user u, 
        masking 'seen_items' if provided.
        """
        scores = self.X[u, :] @ self.Y.T # score vector for affinity of user u to all items

        if seen_items is not None and seen_items.size > 0:
            scores[seen_items] = -np.inf # mask seen items

        topk = np.argsort(-scores)[:K] # get top-K indices
        return topk # return top-K item indices
    

    def full_scores(self) -> np.ndarray:
        """Compute dense score matrix R: X @ Y^T ."""
        return self.X @ self.Y.T
    

    def explain(self, u: int, i: int, train_counts: sparse.csr_matrix, top_m: int = 5):
        """
        Explain why item i is recommended to user u.

        For a target user u and a candidate item i, compute how much each item j
        previously consumed by u contributes to the predicted score p_hat_ui.
        
        For each item j consumed by u, compute contribution s_ij^u * c_uj,
        where s_ij^u = y_i^T W_u y_j, 
        with W_u = A^(-1),
        and A = Y^T C_u Y + reg I.
        
        from: p_hat_ui = sum_{j: r_uj>0} s_ij^u * c_uj.
        """
        # Extract items consumed by user u from CSR
        row_start, row_end = train_counts.indptr[u], train_counts.indptr[u+1]
        item_idx = get_seen_items(train_counts, u) # list of items consumed by user u
        if item_idx.size == 0:
            return []

        # Get raw counts and corresponding confidences
        r_u = train_counts.data[row_start:row_end]
        c_u = self._conf_from_counts(r_u)      
        c_u_minus1 = c_u - 1.0

        # Build A and its inverse W_u
        Y_u = self.Y[item_idx, :]
        YtY = self.Y.T @ self.Y
        I_f = np.eye(self.n_factors, dtype=np.float64)
        A = YtY + (Y_u.T * c_u_minus1) @ Y_u + self.reg * I_f

        W_u = np.linalg.inv(A) # inverse of A

        # Compute contributions s_ij^u for each item j consumed by user u

        y_i = self.Y[i, :][:, None]  # factor of item i

        # s_ij^u = y_i^T W_u y_j  => scalar for each j consumed
        s_ij = (y_i.T @ W_u @ Y_u.T).flatten()  # how much each item j contributes to item i
        contrib = s_ij * c_u                   # weight by confidence

        # Get top-m contributions
        order = np.argsort(-contrib)[:top_m] # indices of top-m contributions
        return [(int(item_idx[j]), float(contrib[j]), float(c_u[j])) for j in order] # return top-m (item, contribution, confidence)


we use now the parameters following the paper: alpha = 40, fact = 64

In [10]:
#=== Train ALS (linear confidence) ===

np.random.seed(RNG_SEED)
als_linear = ImplicitALS(
    n_users=n_users,
    n_items=n_items,
    n_factors=64, # latent factors
    n_iters=10,   # number of ALS iterations
    reg=0.1,      # regularization factor
    alpha=40.0,   # confidence scaling
    use_log_conf=False, # log-based confidence
)
als_linear.fit(train_matrix)


**Metrics evaluation**

We define functions to compute ranking-based metrics also for the ALS model:  

In [11]:
def recall_at_k(model: ImplicitALS, train_csr: sparse.csr_matrix, test_csr: sparse.csr_matrix, K: int = 10) -> float:
    """
    Compute micro-averaged Recall@K for an implicit ALS model.

    For each user u:
      1) Collect test items (ground-truth positives).
      2) Generate top-K recommendations from the model, masking items seen in train.
      3) Count hits and aggregate over all users with non-empty test.
    """ 
    hits, total = 0, 0 # hit count and total relevant items
    for u in range(model.n_users):

        test_items = get_seen_items(test_csr, u)
        if test_items.size == 0:
            continue # skip users with no test items

        seen = get_seen_items(train_csr, u) # items seen in train for user u
        recs = model.recommend_for_user(u, K=K, seen_items=seen) # recommended k items not seen
        hits += len(set(recs).intersection(set(test_items)))
        total += len(test_items)
    return hits / total if total > 0 else np.nan

def average_precision_k(actual: list[int], predicted: list[int], k: int = 10) -> float:
    """
    Compute Average Precision at K (AP@K) for a single user.
    actual: list of ground truth relevant item indices
    predicted: list of predicted item indices (ordered by relevance)
    k: consider only the top-k predictions
    Returns AP@K score.
    """

    # Normalize types
    if isinstance(actual, np.ndarray):
        actual_set = set(int(a) for a in actual.tolist())
    else:
        actual_set = set(int(a) for a in actual)
    if len(actual_set) == 0:  # <-- niente "if actual"
        return 0.0

    # Convert predicted to list of ints and truncate to k
    pred_list = list(int(p) for p in (predicted.tolist() if isinstance(predicted, np.ndarray) else predicted))
    if len(pred_list) > k:
        pred_list = pred_list[:k]

    score, hits = 0.0, 0.0
    seen = set()
    for i, p in enumerate(pred_list, start=1):
        if p in actual_set and p not in seen:
            hits += 1.0
            score += hits / i  # precision at i
            seen.add(p)
    return score / min(len(actual_set), k)

def map_at_k(model: ImplicitALS, train_csr: sparse.csr_matrix, test_csr: sparse.csr_matrix, K: int = 10) -> float:
    """"
    Mean Average Precision at K (MAP@K).
    For each user, compute AP@K and average over all users.
    """
    scores, n = 0.0, 0
    for u in range(model.n_users):
        
        test_items = get_seen_items(test_csr, u) # items in test for user u
        if test_items.size == 0:
            continue
        seen = get_seen_items(train_csr, u)
        recs = model.recommend_for_user(u, K=K, seen_items=seen).tolist()
        scores += average_precision_k(test_items, recs, k=K)
        n += 1
    return scores / n if n > 0 else np.nan

def expected_percentile_ranking(model: ImplicitALS, train_csr: sparse.csr_matrix, test_csr: sparse.csr_matrix) -> float:
    """
    Compute Expected Percentile Ranking (EPR).
    For each user u and each test item i ∈ Test_u, compute the percentile rank
    of i within the full ranking of all items scored for u (0% = best, 100% = worst),
    after masking training-seen items to avoid trivial ranks. 
    Return the mean over all (u, i) pairs. 
    """
    scores = model.full_scores()  # (U, I)
    # Mask seen items to avoid trivial ranks
    for u in range(model.n_users):
        seen = get_seen_items(train_csr, u)
        if seen.size:
            scores[u, seen] = -np.inf 

    # For each positive (u,i) in test, compute percentile rank
    ranks = []
    n_items = scores.shape[1]
    for u in range(model.n_users):
        s = scores[u, :]
        if not np.isfinite(s).any(): # all -inf (cold user)
            continue

        order = np.argsort(-s)  # descending
        inv_rank = np.empty_like(order)
        inv_rank[order] = np.arange(n_items)  # 0 = best
        items = get_seen_items(test_csr, u)

        for i in items:
            r = inv_rank[i] / (n_items - 1) * 100.0  # percentile
            ranks.append(r)
    return float(np.mean(ranks)) if ranks else np.nan

In [12]:
print(f"[ALS (linear)] Recall@{K}: {recall_at_k(als_linear, train_matrix, test_matrix, K):.4f}")
print(f"[ALS (linear)] MAP@{K}:    {map_at_k(als_linear, train_matrix, test_matrix, K):.4f}")
print(f"[ALS (linear)] EPR:        {expected_percentile_ranking(als_linear, train_matrix, test_matrix):.2f}%")

[ALS (linear)] Recall@10: 0.0063
[ALS (linear)] MAP@10:    0.0052
[ALS (linear)] EPR:        22.73%


with this parameters ALS (linear) does not outperforms popularity and item–item CF in Recall@10. We'll see then with parametry sensitivity if improve.

However, EPR is well below 50% (random), showing good personalization and ranking quality.

## 8. Example: Human-readable Recommendations and Explanations

We inspect recommendations for a sample user:

- **Top-K recommendations** are shown with artist names 
- The `explain` function analyzes the **top recommendation**, showing how previously consumed items contribute to its score.  

This provides interpretability: we can see not only *what* is recommended, but also *why* the model ranked it highly.

In [13]:
# === Explain top recommendation for one user (human-readable) ===

def topk_recs_with_names(model: ImplicitALS, u: int, K: int, train_csr, idx2item, item_name_lookup: dict):
    """Return list of (item_index, item_name) for top-K recommendations."""
    seen = get_seen_items(train_csr, u)
    recs = model.recommend_for_user(u, K=K, seen_items=seen)
    names = []
    for i in recs:
        item_id = idx2item[i]
        names.append(item_name_lookup.get(item_id, str(item_id)))
    return list(zip(recs, names))

# Build a stable item_id -> item_name lookup from listens (most frequent name per item)
name_map = (
    listens.groupby(["item_id", "item_name"])
           .size()
           .reset_index(name="cnt")
           .sort_values(["item_id", "cnt"], ascending=[True, False])
           .drop_duplicates(subset=["item_id"])
           .set_index("item_id")["item_name"]
           .to_dict()
)

u_demo = 242     # change this index to inspect a different user
K_demo = 5

recs = topk_recs_with_names(als_linear, u_demo, K_demo, train_matrix, idx2item, name_map)
print(f"User idx {u_demo} ({idx2user[u_demo]}) — Top-{K_demo} recommendations:")
for i, nm in recs:
    print(f"  {i:5d}  {nm}")

# Explain top recommendation
explanations = als_linear.explain(u_demo, recs[0][0], train_matrix, top_m=5)
print(f"\nExplanations for top recommendation (item idx {recs[0][0]} - {recs[0][1]}):\n")

for item_j, contrib, conf in explanations:
    item_name = name_map.get(idx2item[item_j], str(idx2item[item_j]))
    print(f"  Item idx {item_j} ({item_name}): contribution={contrib:.4f}, confidence={conf:.4f}")


User idx 242 (user_000243) — Top-5 recommendations:
  45567  Bob Sinclar
  10048  808 State
  10930  Four Tet
  28221  Bonobo
  37343  Adele

Explanations for top recommendation (item idx 45567 - Bob Sinclar):

  Item idx 8840 (David Guetta): contribution=0.2968, confidence=881.0000
  Item idx 24939 (Paranormal Attack): contribution=0.1706, confidence=1161.0000
  Item idx 22244 (Deepest Blue): contribution=0.1456, confidence=1121.0000
  Item idx 44621 (Jamiroquai): contribution=0.1257, confidence=601.0000
  Item idx 30888 (Simply Red): contribution=0.1090, confidence=881.0000


The model recommends artists like *Bob Sinclair*.  
The explanation indicates that this choice is influenced by past listens (e.g., *David Guetta*, *Paranormal Attack*, ..),  
with contributions weighted by both **confidence** (listening frequency) and **latent similarity** between items.

## 9. ALS vs ALS-LOG (comparison)

We now compare the **linear confidence** version of ALS (as in the paper) with the **log-scaled variant** (optional extension).  

- **ALS (linear)**: \(c_{ui} = 1 + \alpha r_{ui}\)  
- **ALS-LOG**: \(c_{ui} = 1 + \alpha \log(1 + r_{ui}/c_0)\)  

The log-scaled version reduces the dominance of very frequent interactions and provides a robustness check for the results.  

In [14]:
# Train ALS (log confidence)

np.random.seed(RNG_SEED)
als_log = ImplicitALS(
    n_users=n_users,
    n_items=n_items,
    n_factors=64,
    n_iters=10,
    reg=0.1,
    alpha=40.0,
    use_log_conf=True,
)
als_log.fit(train_matrix)

# Evaluate both
def evaluate_model(name, model, train_mat, test_mat, K=10):
    return {
        "Model": name,
        f"Recall@{K}": f"{recall_at_k(model, train_mat, test_mat, K):.4f}",
        f"MAP@{K}":    f"{map_at_k(model,    train_mat, test_mat, K):.4f}",
        "EPR (%)":     f"{expected_percentile_ranking(model, train_mat, test_mat):.2f}"
    }

K = 10
results = [
    evaluate_model("ALS (linear)", als_linear, train_matrix, test_matrix, K),
    evaluate_model("ALS (log)",    als_log,    train_matrix, test_matrix, K)
]

als_compare_df = pd.DataFrame(results)
display(als_compare_df)

,Model,Recall@10,MAP@10,EPR (%)
0,ALS (linear),0.0063,0.0052,22.73
1,ALS (log),0.0088,0.0103,21.00


 Compared to the linear version, ALS with log-scaled confidence achieves slightly more stable results, reducing the dominance of very frequent interactions while maintaining ranking quality.

## 10. Parameter Sensitivity

We vary **α (confidence scaling)** and **latent factors (f)** to test ALS robustness.  

- **α (alpha)**: scales confidence weights, controlling the impact of frequent interactions.  
- **Number of latent factors (f)**: dimensionality of user/item embeddings.

This follows the original paper, which highlights sensitivity mainly to α and f.

In [15]:
def sweep_params(alphas=(10, 40, 100), factors=(16, 32), K=10, use_log_conf=False):
    """
    Grid-search over (alpha, n_factors) for ImplicitALS and collect multiple metrics.
    Reuses `evaluate_model` to compute Recall@K, MAP@K, and EPR in one pass.
    """
    rows = []
    for a in alphas:
        for f in factors:
            np.random.seed(RNG_SEED)
            m = ImplicitALS(
                n_users=n_users,
                n_items=n_items,
                n_factors=f,
                n_iters=10,
                reg=0.1,
                alpha=a,
                use_log_conf=use_log_conf,
            )
            m.fit(train_matrix)

            # Reuse the existing helper to compute all metrics
            row = evaluate_model(
                name=f"ALS ({'log' if use_log_conf else 'linear'})",
                model=m,
                train_mat=train_matrix,
                test_mat=test_matrix,
                K=K
            )
            # Add the hyperparameters as explicit columns
            row.update({"alpha": a, "factors": f})
            rows.append(row)
    
    # Build a tidy DataFrame and order columns nicely
    df = pd.DataFrame(rows)

    # Reorder columns: Model → alpha → factors → metrics
    metric_cols = [f"Recall@{K}", f"MAP@{K}", "EPR (%)"]
    ordered_cols = ["Model", "alpha", "factors"] + metric_cols
    df = df[ordered_cols].sort_values(["Model", "alpha", "factors"]).reset_index(drop=True)       

    return df

sweep_linear = sweep_params(use_log_conf=False, K=10)
display(sweep_linear)

sweep_log = sweep_params(use_log_conf=True, K=10)
display(sweep_log)


,Model,alpha,factors,Recall@10,MAP@10,EPR (%)
0,ALS (linear),10,16,0.0088,0.0072,16.20
1,ALS (linear),10,32,0.0151,0.0211,17.55
2,ALS (linear),40,16,0.0044,0.0050,17.73
3,ALS (linear),40,32,0.0107,0.0082,19.61
4,ALS (linear),100,16,0.0019,0.0008,19.77
5,ALS (linear),100,32,0.0038,0.0033,21.57


,Model,alpha,factors,Recall@10,MAP@10,EPR (%)
0,ALS (log),10,16,0.0182,0.0214,15.33
1,ALS (log),10,32,0.0182,0.0218,16.75
2,ALS (log),40,16,0.0157,0.0219,15.47
3,ALS (log),40,32,0.0157,0.0161,17.05
4,ALS (log),100,16,0.0126,0.0146,15.45
5,ALS (log),100,32,0.0126,0.0153,17.47


Log-scaled confidence proves more robust than linear, with higher Recall/MAP and lower EPR.  
The best setting for ALS is `alpha=10, n_factors=16`. 

Unlike the original paper (which favors larger α and dimensions on big datasets), smaller values perform better here due to the reduced sample size.  
  
- **Lower α** works better, since high values would overweight the few repeated listens.  
- **Fewer latent factors (f=16)** are sufficient, as higher complexity leads to overfitting. 

## 11. Model Comparison: ALS vs Item–Item vs Popularity

We compare the three recommenders side by side at a fixed cutoff (**K = 10**), 
reporting **Recall@10**, **MAP@10**, and **EPR**.

In [16]:
# === Unified comparison at fixed K (K=10) ===

def popularity_metrics(train_csr, test_csr, K=10):
    """Compute Recall@K, MAP@K and EPR for the global popularity baseline."""

    # Global popularity scores (same for all users)
    item_pop = np.asarray(train_csr.sum(axis=0)).ravel()

    hits, total = 0, 0
    ap_sum, n_users_eval = 0.0, 0
    epr_vals = []

    n_items = item_pop.shape[0]
    # Pre-sort once by popularity (desc)
    popular_order = np.argsort(-item_pop)

    for u in range(train_csr.shape[0]):
        test_items = get_seen_items(test_csr, u)
        if test_items.size == 0:
            continue

        seen = set(get_seen_items(train_csr, u).tolist())

        # --- top-K recs: most popular not seen
        recs = [i for i in popular_order if i not in seen][:K]

        # Recall@K
        hits += len(set(recs).intersection(set(test_items.tolist())))
        total += len(test_items)

        # MAP@K
        ap_sum += average_precision_k(actual=test_items.tolist(), predicted=recs, k=K)
        n_users_eval += 1

        # EPR: percentile ranks under popularity scores with masking
        # Build ranks once per user after masking seen
        scores_u = item_pop.copy()
        if seen:
            scores_u[list(seen)] = -np.inf
        order = np.argsort(-scores_u)  # descending
        inv_rank = np.empty_like(order)
        inv_rank[order] = np.arange(n_items)  # 0=best
        denom = max(n_items - 1, 1)
        for i in test_items:
            epr_vals.append(inv_rank[i] / denom * 100.0)

    recall = (round(hits / total, 4)) if total > 0 else np.nan
    mapk = (round(ap_sum / n_users_eval, 4)) if n_users_eval > 0 else np.nan
    epr = float((np.mean(epr_vals).round(2))) if epr_vals else np.nan
    return recall, mapk, epr


def item_item_metrics(S, train_csr, test_csr, K=10):
    """Compute Recall@K, MAP@K and EPR for item–item cosine CF given S (I x I)."""
    import numpy as np

    hits, total = 0, 0
    ap_sum, n_users_eval = 0.0, 0
    epr_vals = []

    n_items = S.shape[0]

    for u in range(train_csr.shape[0]):
        seen = get_seen_items(train_csr, u)
        test_items = get_seen_items(test_csr, u)
        if test_items.size == 0:
            continue

        # Scores by sum of similarities to seen items
        prof = np.zeros(n_items, dtype=np.float64)
        if seen.size > 0:
            prof[seen] = 1.0
        scores = prof @ S
        if seen.size > 0:
            scores[seen] = -np.inf

        # Top-K
        if np.isfinite(scores).any():
            rec_idx = np.argpartition(-scores, min(K, n_items - 1))[:K]
            recs = rec_idx[np.argsort(-scores[rec_idx])].tolist()
        else:
            recs = []

        # Recall@K
        hits += len(set(recs).intersection(set(test_items.tolist())))
        total += len(test_items)

        # MAP@K
        ap_sum += average_precision_k(actual=test_items.tolist(), predicted=recs, k=K)
        n_users_eval += 1

        # EPR
        order = np.argsort(-scores)
        inv_rank = np.empty_like(order)
        inv_rank[order] = np.arange(n_items)
        denom = max(n_items - 1, 1)
        for i in test_items:
            epr_vals.append((inv_rank[i] / denom * 100.0))

    recall = (round(hits / total, 4)) if total > 0 else np.nan
    mapk = (round(ap_sum / n_users_eval, 4)) if n_users_eval > 0 else np.nan
    epr = float((np.mean(epr_vals).round(2))) if epr_vals else np.nan
    return recall, mapk, epr


def evaluate_all_models(
    sweep_linear_df,
    sweep_log_df,
    K=10,
    S=None
):
    """
    Build the unified comparison table at fixed K by:
      - computing baselines (Popularity, Item–Item)
      - concatenating precomputed ALS sweeps (linear, log)
    """

    # 1) Popularity (no hyperparams)
    pop_recall, pop_map, pop_epr = popularity_metrics(train_matrix, test_matrix, K=K)
    base_rows = [{
        "Model": "Popularity",
        "alpha": None, "factors": None,
        f"Recall@{K}": pop_recall, f"MAP@{K}": pop_map, "EPR (%)": pop_epr
    }]

    # 2) Item–Item Cosine (no hyperparams)
    if S is None:
        S = item_item_cosine_scores(train_matrix)
    cos_recall, cos_map, cos_epr = item_item_metrics(S, train_matrix, test_matrix, K=K)
    
    base_rows.append({
        "Model": "Item–Item (cosine)",
        "alpha": None, "factors": None,
        f"Recall@{K}": cos_recall, f"MAP@{K}": cos_map, "EPR (%)": cos_epr
    })

    base_df = pd.DataFrame(base_rows)

    # Ensure consistent column order
    metric_cols = [f"Recall@{K}", f"MAP@{K}", "EPR (%)"]
    ordered_cols = ["Model", "alpha", "factors"] + metric_cols
    
    # Concatenate baselines + ALS 
    all_df = pd.concat(
        [base_df[ordered_cols], sweep_linear_df[ordered_cols], sweep_log_df[ordered_cols]],
        ignore_index=True
    ).sort_values(["Model", "alpha", "factors"]).reset_index(drop=True)

    return all_df


# === Run the unified evaluation (K fixed) ===
K_fixed = 10
all_models_df = evaluate_all_models(
    sweep_linear,
    sweep_log,
    K=K_fixed,
    S=S
)
display(all_models_df)



,Model,alpha,factors,Recall@10,MAP@10,EPR (%)
0,ALS (linear),10,16,0.0088,0.0072,16.20
1,ALS (linear),10,32,0.0151,0.0211,17.55
2,ALS (linear),40,16,0.0044,0.0050,17.73
3,ALS (linear),40,32,0.0107,0.0082,19.61
4,ALS (linear),100,16,0.0019,0.0008,19.77
5,ALS (linear),100,32,0.0038,0.0033,21.57
6,ALS (log),10,16,0.0182,0.0214,15.33
7,ALS (log),10,32,0.0182,0.0218,16.75
8,ALS (log),40,16,0.0157,0.0219,15.47
9,ALS (log),40,32,0.0157,0.0161,17.05


### Conclusion

Results highlight how data sparsity strongly impacts model performance on the reduced Last.fm sample (~500 users, ~2 Million interactions).  

**ALS with log confidence** achieves the best balance: it provides the lowest EPR values, meaning more stable and consistent rankings across the catalog, while also reaching the highest Recall@10 and MAP@10 among ALS variants.  

**ALS with linear confidence** performs worse overall, struggling to capture useful patterns under limited data.  

Interestingly, **Item–Item CF** remains competitive, with Recall and MAP comparable to ALS, confirming that neighborhood methods can still perform well when the dataset is small and sparse.  
Finally, **Popularity** represents the weakest baseline, as expected, since it lacks personalization and is biased towards the most frequent artists.  

Overall, ALS shows its advantage in terms of ranking quality (lower EPR), but Item–Item CF can still match or outperform it on top-K accuracy due to the limited scale and density of the dataset.


**Next Step:** 

rerun on a larger dataset to confirm ALS dominance across all metrics, as shown in the original paper.